In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
pd.set_option('display.max_columns', None)

In [25]:
food_imports = gpd.read_file('./output-data/food-imports-ports.geojson')
unscaled_full_production = gpd.read_file('./output-data/full-production-2017.csv')
unscaled_grown_production = pd.read_csv('./output-data/unscaled-grown-production.csv')
scaled_production = gpd.read_file('./output-data/county-population-consumption-production-scaled.geojson')
ratios = {'Corn': 174.84829232966663, 'Rice': 5.156783499461673, 'Peanuts': 3.15457044814735, 'Sweet corn': 1.636987511629199, 'Wheat': 3.281916724758619, 'Oats': 1.7329857194135225, 'Potatoes': 2.931791059989706, 'Onions': 2.8503840117764994, 'Cucumbers': 3.115981632235308, 'Green peas': 5.416553229173735, 'Tomatoes': 2.5560519152548813, 'Apples': 1.5020456529990078, 'Grapes': 6.4820982243570855, 'Other citrus fruits': 2.8528488412981896, 'Carrots': 2.4943493694156884, 'Oranges': 1.584587706886048, 'Peppers': 0.15375932413546345, 'Lettuce': 2.0750735624121233, 'Cabbage': 1.4767277311642333, 'Celery': 0.1834709173570575, 'Radishes': 0.04746075793112545, 'Tree nuts': 3.3160916577446526, 'Other grain': 0.7188113045533738, 'Broccoli and cauliflower': 1.5729025980419507, 'Stone fruits': 0.3209874536828098, 'Berries': 0.2566350685815907, 'Legumes': 1.3642847984977902, 'Melons': 5.240491816440172, 'Bananas': 2.8148557245952683, 'Tropical fruits': 3.1019642062498285}

In [26]:
appdat = pd.read_csv('./input-data/climate-impact/appdat.csv')
loss = pd.read_csv('./input-data/climate-impact/county_loss.csv')
loss_id = pd.read_csv('./input-data/climate-impact/county_loss_id.csv')


## General Approach:
$$\frac{{(\text{{unscaled\_grown\_production}} \times \text{{climate\_impact}}) + \text{{imported\_production}}}}{{\text{{ratio}}}} = \text{{climate\_stressed\_production}}$$

`unscaled_grown_production` = the number of calories of a crop grown in a county before it is scaled to consumption.  
`climate_impact` = the heat or drought impact to a crop in a county.  
`imported_production` = the number of calories of a crop imported into a county.  
`ratio` = the ratio of produced food to consumed food in a county. This is the number that is used to scale production to consumption.  
`climate_stressed_production` = The calories of production in a county in year of heat stress or climate stress.

In [27]:
# drop appdat Unnamed: 0 column
appdat.drop('Unnamed: 0', axis=1, inplace=True)

In [12]:
appdat

,state,county,crop,average,event,hyp_loss
0,CA,Alameda,Tomatoes,-1.9,Drought,-2.28
1,CA,Alpine,Tomatoes,-6.8,Drought,-8.16
2,CA,Amador,Tomatoes,-7.0,Drought,-8.40
3,CA,Butte,Tomatoes,-3.8,Drought,-4.56
4,CA,Calaveras,Tomatoes,-6.7,Drought,-8.04
...,...,...,...,...,...,...
54750,TX,Wood,Tomatoes,-7.1,Extreme heat,-8.52
54751,TX,Yoakum,Tomatoes,-1.9,Extreme heat,-2.28
54752,TX,Young,Tomatoes,-15.6,Extreme heat,-18.72
54753,TX,Zapata,Tomatoes,-7.7,Extreme heat,-9.24


In [13]:
unscaled_full_production

,geoid,statefp,countyfp,countyns,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,pixels_Corn flour,pixels_Rice dried,pixels_Peanuts,pixels_Sweet corn,pixels_Wheat flour,pixels_Oat flour,pixels_Potatoes,pixels_Onions,pixels_Cucumbers,pixels_Green peas,pixels_Tomatoes,pixels_Apples,pixels_Grapes,pixels_Other citrus fruits,pixels_Carrots,pixels_Oranges,pixels_Peppers,pixels_Lettuce,pixels_Cabbage,pixels_Celery,pixels_Radishes,state,postal,kcal_Corn flour,kg_Corn flour,kcal_Rice dried,kg_Rice dried,kcal_Peanuts,kg_Peanuts,kcal_Sweet corn,kg_Sweet corn,kcal_Wheat flour,kg_Wheat flour,kcal_Oat flour,kg_Oat flour,kcal_Potatoes,kg_Potatoes,kcal_Onions,kg_Onions,kcal_Cucumbers,kg_Cucumbers,kcal_Green peas,kg_Green peas,kcal_Tomatoes,kg_Tomatoes,kcal_Apples,kg_Apples,kcal_Grapes,kg_Grapes,kcal_Other citrus fruits,kg_Other citrus fruits,kcal_Carrots,kg_Carrots,kcal_Oranges,kg_Oranges,kcal_Peppers,kg_Peppers,kcal_Lettuce,kg_Lettuce,kcal_Cabbage,kg_Cabbage,kcal_Celery,kg_Celery,kcal_Radishes,kg_Radishes,pixels_Tree nuts,kcal_Tree nuts,kg_Tree nuts,pixels_Other grain,kcal_Other grain,kg_Other grain,pixels_Broccoli and cauliflower,kcal_Broccoli and cauliflower,kg_Broccoli and cauliflower,pixels_Stone fruits,kcal_Stone fruits,kg_Stone fruits,pixels_Berries,kcal_Berries,kg_Berries,pixels_Legumes,kcal_Legumes,kg_Legumes,pixels_Melons,kcal_Melons,kg_Melons,pixels_Bananas,kcal_Bananas,kg_Bananas,pixels_Tropical fruits,kcal_Tropical fruits,kg_Tropical fruits,geometry
0,31039,31,39,835841,Cuming,Cuming County,6,H1,G4020,0.0,0.0,0.0,A,1477645345,10690204,41.9158651,-96.7885168,649877,0,0,0,364,848,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,2558898985033.2075,701068735.5084152,0.0,0.0,0.0,0.0,0.0,0.0,373858736.43211734,112000.90664756975,271670865.580192,69838.26878668176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,None
1,31109,31,109,835876,Lancaster,Lancaster County,6,H1,G4020,339.0,30700.0,0.0,A,2169272970,22847034,40.7835474,-96.6886584,557931,0,0,0,4476,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,2196860436080.3083,601880018.328,0.0,0.0,0.0,0.0,0.0,0.0,4597229956.786146,1377241.918006929,108283906.32795389,27836.479775823624,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,439,124661310.54732545,36882.044540628835,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,None
2,31129,31,129,835886,Nuckolls,Nuckolls County,6,H1,G4020,0.0,0.0,0.0,A,1489645185,1718484,40.1764918,-98.0468422,492281,0,0,0,51658,1569,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,1938362722870.8394,531058674.46427274,0.0,0.0,0.0,0.0,0.0,0.0,53057128040.13824,15894897.900000434,502655174.64070904,129217.26854516943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,None
3,31101,31,101,835872,Keith,Keith County,6,H1,G4020,0.0,0.0,0.0,A,2749903240,124629167,41.1942448,-101.6444493,489931,0,0,0,130044,7521,0,0,0,7954,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,1929109567866.3877,528523561.6222353,0.0,0.0,0.0,0.0,0.0,0.0,133566169012.57767,40013862.3738367,2409477099.0903587,619402.8532365961,0.0,0.0,0.0,0.0,0.0,0.0,4268627887.0917025,1629247.2851495047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,4131,1261239370.0991397,371550.14703634515,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,4779,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,None
4,31137,31,137,835890,Phelps,Phelps County,6,H1,G4020,0.0,0.0,0.0,A,1398048634,1646526,40.5163647,-99.4065568,719100,0,0,0,7193,757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,2831465431362.211,775744529.6634616,0.0,0.0,0.0,0.0,0.0,0.0,7387818382.297308,2213248.685483432,242517506.18420443,62343.8319

In [14]:
unscaled_grown_production

,statefp,countyfp,countyns,geoid,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,pixels_Corn flour,pixels_Rice dried,pixels_Peanuts,pixels_Sweet corn,pixels_Wheat flour,pixels_Oat flour,pixels_Potatoes,pixels_Onions,pixels_Cucumbers,pixels_Green peas,pixels_Tomatoes,pixels_Apples,pixels_Grapes,pixels_Other citrus fruits,pixels_Carrots,pixels_Oranges,pixels_Peppers,pixels_Lettuce,pixels_Cabbage,pixels_Celery,pixels_Radishes,state,postal,kcal_Corn flour,kg_Corn flour,kcal_Rice dried,kg_Rice dried,kcal_Peanuts,kg_Peanuts,kcal_Sweet corn,kg_Sweet corn,kcal_Wheat flour,kg_Wheat flour,kcal_Oat flour,kg_Oat flour,kcal_Potatoes,kg_Potatoes,kcal_Onions,kg_Onions,kcal_Cucumbers,kg_Cucumbers,kcal_Green peas,kg_Green peas,kcal_Tomatoes,kg_Tomatoes,kcal_Apples,kg_Apples,kcal_Grapes,kg_Grapes,kcal_Other citrus fruits,kg_Other citrus fruits,kcal_Carrots,kg_Carrots,kcal_Oranges,kg_Oranges,kcal_Peppers,kg_Peppers,kcal_Lettuce,kg_Lettuce,kcal_Cabbage,kg_Cabbage,kcal_Celery,kg_Celery,kcal_Radishes,kg_Radishes,pixels_Tree nuts,kcal_Tree nuts,kg_Tree nuts,pixels_Other grain,kcal_Other grain,kg_Other grain,pixels_Broccoli and cauliflower,kcal_Broccoli and cauliflower,kg_Broccoli and cauliflower,pixels_Stone fruits,kcal_Stone fruits,kg_Stone fruits,pixels_Berries,kcal_Berries,kg_Berries,pixels_Legumes,kcal_Legumes,kg_Legumes,pixels_Melons,kcal_Melons,kg_Melons,pixels_Bananas,kcal_Bananas,kg_Bananas,pixels_Tropical fruits,kcal_Tropical fruits,kg_Tropical fruits
0,31,39,835841,31039,Cuming,Cuming County,6,H1,G4020,NaN,NaN,NaN,A,1477645345,10690204,41.915865,-96.788517,649877,0,0,0,364,848,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,2.558899e+12,7.010687e+08,NaN,NaN,NaN,NaN,NaN,NaN,3.738587e+08,1.120009e+05,2.716709e+08,69838.268787,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0,0,0,0,0
1,31,109,835876,31109,Lancaster,Lancaster County,6,H1,G4020,339.0,30700.0,NaN,A,2169272970,22847034,40.783547,-96.688658,557931,0,0,0,4476,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,2.196860e+12,6.018800e+08,NaN,NaN,NaN,NaN,NaN,NaN,4.597230e+09,1.377242e+06,1.082839e+08,27836.479776,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,439,1.246613e+08,36882.044541,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0,0,0,0,0
2,31,129,835886,31129,Nuckolls,Nuckolls County,6,H1,G4020,NaN,NaN,NaN,A,1489645185,1718484,40.176492,-98.046842,492281,0,0,0,51658,1569,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,1.938363e+12,5.310587e+08,NaN,NaN,NaN,NaN,NaN,NaN,5.305713e+10,1.589490e+07,5.026552e+08,129217.268545,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0,0,0,0,0
3,31,101,835872,31101,Keith,Keith County,6,H1,G4020,NaN,NaN,NaN,A,2749903240,124629167,41.194245,-101.644449,489931,0,0,0,130044,7521,0,0,0,7954,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,1.929110e+12,5.285236e+08,NaN,NaN,NaN,NaN,NaN,NaN,1.335662e+11,4.001386e+07,2.409477e+09,619402.853237,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,4.268628e+09,1.629247e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4131,1.261239e+09,371550.147036,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,4779,NaN,NaN,0,NaN,NaN,0,0,0,0,0,0
4,31,137,835890,31137,Phelps,Phelps County,6,H1,G4020,NaN,NaN,NaN,A,1398048634,1646526,40.516365,-99.406557,719100,0,0,0,7193,757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,2.831465e+12,7.757445e+08,NaN,NaN,NaN,NaN,NaN,NaN,7.387818e+09,2.213249e+06,2.425175e+08,62343.831924,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,N

In [28]:
# Select columns that start with 'kg' or 'kcal' from unscaled_grown_production
relevant_columns = [col for col in unscaled_grown_production.columns if col.startswith('kg') or col.startswith('kcal')]

unscaled_full_production[relevant_columns] = unscaled_full_production[relevant_columns].astype('float64')
# # Create a new dataframe to store the results
unscaled_import_production = unscaled_full_production.copy()

# Perform subtraction for selected columns
unscaled_import_production[relevant_columns] = unscaled_full_production[relevant_columns] - unscaled_grown_production[relevant_columns]

unscaled_import_production


,geoid,statefp,countyfp,countyns,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,pixels_Corn flour,pixels_Rice dried,pixels_Peanuts,pixels_Sweet corn,pixels_Wheat flour,pixels_Oat flour,pixels_Potatoes,pixels_Onions,pixels_Cucumbers,pixels_Green peas,pixels_Tomatoes,pixels_Apples,pixels_Grapes,pixels_Other citrus fruits,pixels_Carrots,pixels_Oranges,pixels_Peppers,pixels_Lettuce,pixels_Cabbage,pixels_Celery,pixels_Radishes,state,postal,kcal_Corn flour,kg_Corn flour,kcal_Rice dried,kg_Rice dried,kcal_Peanuts,kg_Peanuts,kcal_Sweet corn,kg_Sweet corn,kcal_Wheat flour,kg_Wheat flour,kcal_Oat flour,kg_Oat flour,kcal_Potatoes,kg_Potatoes,kcal_Onions,kg_Onions,kcal_Cucumbers,kg_Cucumbers,kcal_Green peas,kg_Green peas,kcal_Tomatoes,kg_Tomatoes,kcal_Apples,kg_Apples,kcal_Grapes,kg_Grapes,kcal_Other citrus fruits,kg_Other citrus fruits,kcal_Carrots,kg_Carrots,kcal_Oranges,kg_Oranges,kcal_Peppers,kg_Peppers,kcal_Lettuce,kg_Lettuce,kcal_Cabbage,kg_Cabbage,kcal_Celery,kg_Celery,kcal_Radishes,kg_Radishes,pixels_Tree nuts,kcal_Tree nuts,kg_Tree nuts,pixels_Other grain,kcal_Other grain,kg_Other grain,pixels_Broccoli and cauliflower,kcal_Broccoli and cauliflower,kg_Broccoli and cauliflower,pixels_Stone fruits,kcal_Stone fruits,kg_Stone fruits,pixels_Berries,kcal_Berries,kg_Berries,pixels_Legumes,kcal_Legumes,kg_Legumes,pixels_Melons,kcal_Melons,kg_Melons,pixels_Bananas,kcal_Bananas,kg_Bananas,pixels_Tropical fruits,kcal_Tropical fruits,kg_Tropical fruits,geometry
0,31039,31,39,835841,Cuming,Cuming County,6,H1,G4020,0.0,0.0,0.0,A,1477645345,10690204,41.9158651,-96.7885168,649877,0,0,0,364,848,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,5.960464e-08,-1.455192e-11,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.0,0.000000e+00,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0.0,0,0.0,0.0,None
1,31109,31,109,835876,Lancaster,Lancaster County,6,H1,G4020,339.0,30700.0,0.0,A,2169272970,22847034,40.7835474,-96.6886584,557931,0,0,0,4476,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,1.490116e-08,0.000000e+00,0.0,0.0,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,439,0.0,-7.275958e-12,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0.0,0,0.0,0.0,None
2,31129,31,129,835886,Nuckolls,Nuckolls County,6,H1,G4020,0.0,0.0,0.0,A,1489645185,1718484,40.1764918,-98.0468422,492281,0,0,0,51658,1569,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,-0.000244,-5.960464e-08,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,5.960464e-08,-1.455192e-11,0.0,0.0,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.0,0.000000e+00,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0.0,0.0,0,0.0,0.0,None
3,31101,31,101,835872,Keith,Keith County,6,H1,G4020,0.0,0.0,0.0,A,2749903240,124629167,41.1942448,-101.6444493,489931,0,0,0,130044,7521,0,0,0,7954,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,-0.000244,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,-1.525879e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,NaN,NaN,-9.536743e-07,-2.328306e-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,4131,0.0,-5.820766e-11,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,4779,NaN,NaN,0,NaN,NaN,0,0.0,0.0,0,0.0,0.0,None
4,31137,31,137,835890,Phelps,Phelps County,6,H1,G4020,0.0,0.0,0.0,A,1398048634,1646526,40.5163647,-99.4065568,719100,0,0,0,7193,757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [141]:
# write unscaled_import_production to csv
# unscaled_import_production.to_csv('./output-data/unscaled-import-production.csv')

In [29]:
def lookup_impact(crop, county, event, calories):
    try:
        impact = (appdat[(appdat['county'] == county) & (appdat['crop'] == crop) & (appdat['event'] == event)]['hyp_loss'].values[0])/100
        return (calories+(impact * calories))
    except IndexError:
        return None

In [ ]:
drought_grown_production = unscaled_grown_production.copy()
relevant_columns = [col for col in drought_grown_production.columns if col.startswith('kg') or col.startswith('kcal')]

for col in relevant_columns:
    crop = col.split('_')[1]
    # print (crop)
    drought_grown_production[col] = drought_grown_production.apply(
        lambda row: lookup_impact(crop, row['name'], 'Drought', row[col]),
        axis=1
    )
drought_grown_production

In [ ]:
heat_grown_production = unscaled_grown_production.copy()
relevant_columns = [col for col in heat_grown_production.columns if col.startswith('kg') or col.startswith('kcal')]
for col in relevant_columns:
    crop = col.split('_')[1]
    print (crop)
    heat_grown_production[col] = heat_grown_production.apply(
        lambda row: lookup_impact(crop, row['name'], 'Extreme heat', row[col]),
        axis=1
    )
heat_grown_production

Corn flour
Corn flour
Rice dried
Rice dried
Peanuts
Peanuts
Sweet corn
Sweet corn
Wheat flour
Wheat flour
Oat flour
Oat flour
Potatoes
Potatoes
Onions
Onions
Cucumbers
Cucumbers
Green peas
Green peas
Tomatoes
Tomatoes
Apples
Apples
Grapes
Grapes
Other citrus fruits
Other citrus fruits
Carrots
Carrots
Oranges
Oranges
Peppers
Peppers
Lettuce
Lettuce
Cabbage
Cabbage
Celery
Celery
Radishes
Radishes
Tree nuts
Tree nuts
Other grain
Other grain
Broccoli and cauliflower
Broccoli and cauliflower
Stone fruits
Stone fruits
Berries
Berries
Legumes
Legumes
Melons
Melons
Bananas
Bananas
Tropical fruits
Tropical fruits


,statefp,countyfp,countyns,geoid,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,pixels_Corn flour,pixels_Rice dried,pixels_Peanuts,pixels_Sweet corn,pixels_Wheat flour,pixels_Oat flour,pixels_Potatoes,pixels_Onions,pixels_Cucumbers,pixels_Green peas,pixels_Tomatoes,pixels_Apples,pixels_Grapes,pixels_Other citrus fruits,pixels_Carrots,pixels_Oranges,pixels_Peppers,pixels_Lettuce,pixels_Cabbage,pixels_Celery,pixels_Radishes,state,postal,kcal_Corn flour,kg_Corn flour,kcal_Rice dried,kg_Rice dried,kcal_Peanuts,kg_Peanuts,kcal_Sweet corn,kg_Sweet corn,kcal_Wheat flour,kg_Wheat flour,kcal_Oat flour,kg_Oat flour,kcal_Potatoes,kg_Potatoes,kcal_Onions,kg_Onions,kcal_Cucumbers,kg_Cucumbers,kcal_Green peas,kg_Green peas,kcal_Tomatoes,kg_Tomatoes,kcal_Apples,kg_Apples,kcal_Grapes,kg_Grapes,kcal_Other citrus fruits,kg_Other citrus fruits,kcal_Carrots,kg_Carrots,kcal_Oranges,kg_Oranges,kcal_Peppers,kg_Peppers,kcal_Lettuce,kg_Lettuce,kcal_Cabbage,kg_Cabbage,kcal_Celery,kg_Celery,kcal_Radishes,kg_Radishes,pixels_Tree nuts,kcal_Tree nuts,kg_Tree nuts,pixels_Other grain,kcal_Other grain,kg_Other grain,pixels_Broccoli and cauliflower,kcal_Broccoli and cauliflower,kg_Broccoli and cauliflower,pixels_Stone fruits,kcal_Stone fruits,kg_Stone fruits,pixels_Berries,kcal_Berries,kg_Berries,pixels_Legumes,kcal_Legumes,kg_Legumes,pixels_Melons,kcal_Melons,kg_Melons,pixels_Bananas,kcal_Bananas,kg_Bananas,pixels_Tropical fruits,kcal_Tropical fruits,kg_Tropical fruits
0,31,39,835841,31039,Cuming,Cuming County,6,H1,G4020,NaN,NaN,NaN,A,1477645345,10690204,41.915865,-96.788517,649877,0,0,0,364,848,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,None,0.000000e+00,0.000000,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,None,None,0,None,None
1,31,109,835876,31109,Lancaster,Lancaster County,6,H1,G4020,339.0,30700.0,NaN,A,2169272970,22847034,40.783547,-96.688658,557931,0,0,0,4476,338,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,None,0.000000e+00,0.000000,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,0,NaN,NaN,439,1.159849e+08,34315.054241,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,None,None,0,None,None
2,31,129,835886,31129,Nuckolls,Nuckolls County,6,H1,G4020,NaN,NaN,NaN,A,1489645185,1718484,40.176492,-98.046842,492281,0,0,0,51658,1569,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,None,0.000000e+00,0.000000,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,None,None,0,None,None
3,31,101,835872,31101,Keith,Keith County,6,H1,G4020,NaN,NaN,NaN,A,2749903240,124629167,41.194245,-101.644449,489931,0,0,0,130044,7521,0,0,0,7954,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,None,0.000000e+00,0.000000,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,0,NaN,NaN,4131,1.241564e+09,365753.964743,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,4779,NaN,NaN,0,NaN,NaN,0,None,None,0,None,None
4,31,137,835890,31137,Phelps,Phelps County,6,H1,G4020,NaN,NaN,NaN,A,1398048634,1646526,40.516365,-99.406557,719100,0,0,0,7193,757,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Nebraska,NE,None,None,None,None,NaN,NaN,NaN,NaN,None,None,None,None,0.000000e+00,0.000000,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,None,None,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
# write heat_grown_production and drought_grown_production to csv files

heat_grown_production.to_csv('./output-data/heat_grown_production.csv')
drought_grown_production.to_csv('./output-data/drought_grown_production.csv')

In [30]:
heat_grown_production = pd.read_csv('./output-data/unscaled_heat_grown_production.csv')
drought_grown_production = pd.read_csv('./output-data/unscaled_drought_grown_production.csv')

In [31]:
for c in heat_grown_production.columns:
    if c in relevant_columns:
        crop = c.split('_')[1]
        units = c.split('_')[0]
        new_column_name = f'{units}_heat_{crop}'
        heat_grown_production.rename(columns={c: new_column_name}, inplace=True)

for c in drought_grown_production.columns:
    if c in relevant_columns:
        crop = c.split('_')[1]
        units = c.split('_')[0]
        new_column_name = f'{units}_drought_{crop}'
        drought_grown_production.rename(columns={c: new_column_name}, inplace=True)


In [32]:
column_mapping = {
    'Corn flour': 'Corn',
    'Oat flour': 'Oats',
    'Rice dried': 'Rice',
    'Wheat flour': 'Wheat'
}
def rename_columns(col):
    parts = col.split('_')
    if len(parts) > 2:
        i = ' '.join(parts[2:])  
        if u in column_mapping:
            return f"{parts[0]}_{parts[1]}_{column_mapping[i]}"
    return col

heat_grown_production.columns = heat_grown_production.columns.map(rename_columns)
drought_grown_production.columns = drought_grown_production.columns.map(rename_columns)
unscaled_import_production.columns = unscaled_import_production.columns.map(rename_columns)
unscaled_grown_production.columns = unscaled_grown_production.columns.map(rename_columns)


In [37]:
heat_grown_production.to_csv('./output-data/unscaled-heat-grown-production.csv')
drought_grown_production.to_csv('./output-data/unscaled-drought-grown-production.csv')
unscaled_import_production.to_csv('./output-data/unscaled-import-production.csv')
unscaled_grown_production.to_csv('./output-data/unscaled-grown-production.csv')

## Scaling and joining production data
`scaled_production` is the geodataframe with or current scaled production predictions. In this section we will divide our unscaled numbers by the scale ratios and join to scaled_production with column names including `_heat_` our `_drought_` to denote the drought or heat impacted. 

In [103]:
heat_grown_production = pd.read_csv('output-data/unscaled-heat-grown-production.csv')
drought_grown_production = pd.read_csv('output-data/unscaled-drought-grown-production.csv')
unscaled_import_production = pd.read_csv('output-data/unscaled-import-production.csv')


In [104]:
# in heat_grown_production for any column name that starts with kg or kcal print the column name
def scale_production(df):
    working_df = df.copy()
    for column in working_df.columns:
        if column.startswith('kg') or column.startswith('kcal'):
            # print everything after the last _
            crop_name = column.split('_')[-1]
            working_df[column] = working_df[column] / ratios[crop_name]
    return working_df
scaled_heat_grown_production = scale_production(heat_grown_production)
scaled_drought_grown_production = scale_production(drought_grown_production)
scaled_import_production = scale_production(unscaled_import_production)

In [108]:
# return the row with the index 3103 of the column kcal_heat_potatoes
# ratio is "Potatoes": 2.931791059989706
print(scaled_drought_grown_production.loc[100, 'kcal_drought_Potatoes'])
print(drought_grown_production.loc[100, 'kcal_drought_Potatoes'])
print ("ratio should be 2.931791059989706")
print ("ratio is " + str(drought_grown_production.loc[100, 'kcal_drought_Potatoes']/scaled_drought_grown_production.loc[100, 'kcal_drought_Potatoes']))



158392813.50315592
464374634.5951693
ratio should be 2.931791059989706
ratio is 2.931791059989706


In [112]:
# in heat_grown_production for any column name that starts with kg or kcal print the column name
def add_imports(df):
    working_df = df.copy()
    working_df = working_df.fillna(0)
    for column in working_df.columns:
        if column.startswith('kg'):
            crop_name = column.split('_')[-1]
            working_df[column] = working_df[column] + scaled_import_production["kg_"+crop_name]
        if column.startswith('kcal'):
            crop_name = column.split('_')[-1]
            working_df[column] = working_df[column] + scaled_import_production["kcal_"+crop_name]
    return working_df
scaled_heat_production = add_imports(scaled_heat_grown_production)
scaled_drought_production = add_imports(scaled_drought_grown_production)


In [113]:
# return the row of scaled_import_production with countyfp as 201
scaled_import_production[scaled_import_production['countyfp'] == 201]

,Unnamed: 0,geoid,statefp,countyfp,countyns,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,pixels_Corn,pixels_Rice,pixels_Peanuts,pixels_Sweet corn,pixels_Wheat,pixels_Oats,pixels_Potatoes,pixels_Onions,pixels_Cucumbers,pixels_Green peas,pixels_Tomatoes,pixels_Apples,pixels_Grapes,pixels_Other citrus fruits,pixels_Carrots,pixels_Oranges,pixels_Peppers,pixels_Lettuce,pixels_Cabbage,pixels_Celery,pixels_Radishes,state,postal,kcal_Corn,kg_Corn,kcal_Rice,kg_Rice,kcal_Peanuts,kg_Peanuts,kcal_Sweet corn,kg_Sweet corn,kcal_Wheat,kg_Wheat,kcal_Oats,kg_Oats,kcal_Potatoes,kg_Potatoes,kcal_Onions,kg_Onions,kcal_Cucumbers,kg_Cucumbers,kcal_Green peas,kg_Green peas,kcal_Tomatoes,kg_Tomatoes,kcal_Apples,kg_Apples,kcal_Grapes,kg_Grapes,kcal_Other citrus fruits,kg_Other citrus fruits,kcal_Carrots,kg_Carrots,kcal_Oranges,kg_Oranges,kcal_Peppers,kg_Peppers,kcal_Lettuce,kg_Lettuce,kcal_Cabbage,kg_Cabbage,kcal_Celery,kg_Celery,kcal_Radishes,kg_Radishes,pixels_Tree nuts,kcal_Tree nuts,kg_Tree nuts,pixels_Other grain,kcal_Other grain,kg_Other grain,pixels_Broccoli and cauliflower,kcal_Broccoli and cauliflower,kg_Broccoli and cauliflower,pixels_Stone fruits,kcal_Stone fruits,kg_Stone fruits,pixels_Berries,kcal_Berries,kg_Berries,pixels_Legumes,kcal_Legumes,kg_Legumes,pixels_Melons,kcal_Melons,kg_Melons,pixels_Bananas,kcal_Bananas,kg_Bananas,pixels_Tropical fruits,kcal_Tropical fruits,kg_Tropical fruits,geometry
333,333,48201,48,201,1383886,Harris,Harris County,6,H1,G4020,288.0,26420.0,0.0,A,4421068052,182379558,29.857273,-95.393037,15816,8889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Texas,TX,1.522111e+07,4.193144e+03,1.631612e+10,1.263078e+07,1.908871e+09,325191.025803,NaN,NaN,3.602281e+10,1.085024e+07,0.0,0.0,1.142378e+09,906649.193483,2.993841e+08,654075.378018,1.830815e+08,416094.237074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.388647e+09,6.448771e+06,5.300653e+06,12928.421494,3.033583e+09,6.073516e+06,1.415641e+07,64347.317183,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.0,0.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,2.487056e+09,7.133160e+06,0,7.026923e+10,7.244251e+07,0,3.326967e+10,6.141517e+07,NaN
560,560,21201,21,201,516947,Robertson,Robertson County,6,H1,G4020,0.0,0.0,0.0,A,258794976,484952,38.513473,-84.064230,496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Kentucky,KY,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.0,0.0,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,NaN
942,942,13201,13,201,344088,Miller,Miller County,6,H1,G4020,0.0,0.0,0.0,A,731471756,3211332,31.162931,-84.730398,34484,0,119541,0,2514,821,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Georgia,GA,0.000000e+00,-2.130584e-11,NaN,NaN,-1.934817e-05,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,20261,NaN,NaN,18589,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,380,NaN,NaN,404,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,NaN
1330,1330,29201,29,201,758553,Scott,Scott County,6,H1,G4020,164.0,43460.0,0.0,A,1087808789,15175233,37.047793,-89.568097,234406,7415,669,0,3283,0,1123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Missouri,MO,6.981499e-07,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0.0,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,10095,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,NaN
2398,2398,20201,20,201,485061,Washington,Washington County,6,H1,G4020,0.0,0.0,0.0,A,2317414332,10139473,39.776714,-97.095611,364607,0,0,0,122765,7533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Kans

In [114]:
scaled_heat_production[scaled_heat_production['countyfp'] == 201]

,Unnamed: 0.1,Unnamed: 0,statefp,countyfp,countyns,geoid,name,namelsad,lsad,classfp,mtfcc,csafp,cbsafp,metdivfp,funcstat,aland,awater,intptlat,intptlon,pixels_Corn,pixels_Rice,pixels_Peanuts,pixels_Sweet corn,pixels_Wheat,pixels_Oats,pixels_Potatoes,pixels_Onions,pixels_Cucumbers,pixels_Green peas,pixels_Tomatoes,pixels_Apples,pixels_Grapes,pixels_Other citrus fruits,pixels_Carrots,pixels_Oranges,pixels_Peppers,pixels_Lettuce,pixels_Cabbage,pixels_Celery,pixels_Radishes,state,postal,kcal_heat_Corn,kg_heat_Corn,kcal_heat_Rice,kg_heat_Rice,kcal_heat_Peanuts,kg_heat_Peanuts,kcal_heat_Sweet corn,kg_heat_Sweet corn,kcal_heat_Wheat,kg_heat_Wheat,kcal_heat_Oats,kg_heat_Oats,kcal_heat_Potatoes,kg_heat_Potatoes,kcal_heat_Onions,kg_heat_Onions,kcal_heat_Cucumbers,kg_heat_Cucumbers,kcal_heat_Green peas,kg_heat_Green peas,kcal_heat_Tomatoes,kg_heat_Tomatoes,kcal_heat_Apples,kg_heat_Apples,kcal_heat_Grapes,kg_heat_Grapes,kcal_heat_Other citrus fruits,kg_heat_Other citrus fruits,kcal_heat_Carrots,kg_heat_Carrots,kcal_heat_Oranges,kg_heat_Oranges,kcal_heat_Peppers,kg_heat_Peppers,kcal_heat_Lettuce,kg_heat_Lettuce,kcal_heat_Cabbage,kg_heat_Cabbage,kcal_heat_Celery,kg_heat_Celery,kcal_heat_Radishes,kg_heat_Radishes,pixels_Tree nuts,kcal_heat_Tree nuts,kg_heat_Tree nuts,pixels_Other grain,kcal_heat_Other grain,kg_heat_Other grain,pixels_Broccoli and cauliflower,kcal_heat_Broccoli and cauliflower,kg_heat_Broccoli and cauliflower,pixels_Stone fruits,kcal_heat_Stone fruits,kg_heat_Stone fruits,pixels_Berries,kcal_heat_Berries,kg_heat_Berries,pixels_Legumes,kcal_heat_Legumes,kg_heat_Legumes,pixels_Melons,kcal_heat_Melons,kg_heat_Melons,pixels_Bananas,kcal_heat_Bananas,kg_heat_Bananas,pixels_Tropical fruits,kcal_heat_Tropical fruits,kg_heat_Tropical fruits
333,333,333,48,201,1383886,48201,Harris,Harris County,6,H1,G4020,288.0,26420.0,0.0,A,4421068052,182379558,29.857273,-95.393037,15816,8889,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Texas,TX,1.522111e+07,4.193144e+03,1.631612e+10,1.263078e+07,1.908871e+09,3.251910e+05,NaN,NaN,3.602281e+10,1.085024e+07,0.0,0.0,1.142378e+09,9.066492e+05,2.993841e+08,654075.378018,1.830815e+08,416094.237074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.388647e+09,6.448771e+06,5.300653e+06,12928.421494,3.033583e+09,6.073516e+06,1.415641e+07,64347.317183,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,2.487056e+09,7.133160e+06,0,7.026923e+10,7.244251e+07,0,3.326967e+10,6.141517e+07
560,560,560,21,201,516947,21201,Robertson,Robertson County,6,H1,G4020,0.0,0.0,0.0,A,258794976,484952,38.513473,-84.064230,496,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Kentucky,KY,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00
942,942,942,13,201,344088,13201,Miller,Miller County,6,H1,G4020,0.0,0.0,0.0,A,731471756,3211332,31.162931,-84.730398,34484,0,119541,0,2514,821,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Georgia,GA,0.000000e+00,-2.130584e-11,NaN,NaN,1.274908e+11,2.248515e+07,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0.0,NaN,NaN,0.000000e+00,0.000000,0.000000e+00,0.000000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,20261,NaN,NaN,18589,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN,380,NaN,NaN,404,3.214524e+08,2.060593e+06,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00
1330,1330,1330,29,201,758553,29201,Scott,Scott County,6,H1,G4020,164.0,43460.0,0.0,A,1087808789,15175233,37.047793,-89.568097,234406,7415,669,0,3283,0,1123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Missouri,MO,6.981499e-07,0.000000e+00,0.000000e+00,0.000000e+00,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.0,0.0,1.692149e+10,2.927594e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [116]:
# Drought Check
print(scaled_drought_production.loc[333, 'kcal_drought_Wheat'])
print(scaled_drought_grown_production.loc[333, 'kcal_drought_Wheat'])
print ("first number should be " + str(scaled_import_production.loc[333, 'kcal_Wheat']))
# Heat check
print(scaled_heat_production.loc[333, 'kcal_heat_Wheat'])
print(scaled_heat_grown_production.loc[333, 'kcal_heat_Wheat'])
print ("first number should be " + str(scaled_import_production.loc[333, 'kcal_Wheat']))

36022808814.16795
nan
second number should be 36022808814.16795


In [164]:
for c in scaled_heat_production.columns:
    if c.startswith('kg') or c.startswith('kcal'):
        scaled_production = scaled_production.merge(scaled_heat_production[['geoid', c]], on='geoid', how='left')
for c in scaled_drought_production.columns:
    if c.startswith('kg') or c.startswith('kcal'):
        scaled_production = scaled_production.merge(scaled_drought_production[['geoid', c]], on='geoid', how='left')

In [165]:
for c in scaled_production.columns:
    if c.startswith('kg') or c.startswith('kcal') or c.startswith('other') or c.startswith('total') or c.startswith('low') or c.startswith('high'):
        scaled_production[c] = scaled_production[c].fillna(0)
        scaled_production[c] = scaled_production[c].astype(int)
        scaled_production[c] = scaled_production[c].replace(0, np.nan)

In [166]:
for col in scaled_production.columns:
    if "Wheat flour" in col:
        new_col = col.replace("Wheat flour", "Wheat")
        scaled_production.rename(columns={col: new_col}, inplace=True)
    if "Corn flour" in col:
        new_col = col.replace("Corn flour", "Corn")
        scaled_production.rename(columns={col: new_col}, inplace=True)
    if "Rice dried" in col:
        new_col = col.replace("Rice dried", "Rice")
        scaled_production.rename(columns={col: new_col}, inplace=True)
    if "Oat flour" in col:
        new_col = col.replace("Oat flour", "Oats")
        scaled_production.rename(columns={col: new_col}, inplace=True)
    if "drought" in col:
        new_col = col.replace("drought", "produced_drought")
        scaled_production.rename(columns={col: new_col}, inplace=True)
    if "heat" in col:
        new_col = col.replace("_heat", "_produced_heat")
        scaled_production.rename(columns={col: new_col}, inplace=True)

In [167]:
print(scaled_production["kcal_produced_heat_Wheat"].max())
print(scaled_production["kcal_produced_Wheat"].max())

36022808814.0
885882644040.0


In [162]:
list(scaled_production.columns)

['geoid',
 'geography',
 'geographic_area_name',
 'total_population',
 'agesex_male_total',
 'agesex_female_total',
 'agesex_male_kids',
 'agesex_male_adults',
 'agesex_female_kids',
 'agesex_female_adults',
 'agesex_kid_total',
 'agesex_adult_total',
 'income_total_population_determined',
 'income_low_income',
 'income_high_income',
 'income_unknown_income',
 'race_total_population_determined',
 'race_black-african_american',
 'race_hispanic-latino',
 'race_white',
 'race_other_race',
 'race_unknown_race',
 'statefp',
 'countyfp',
 'countyns',
 'name',
 'namelsad',
 'stusps',
 'state_name',
 'lsad',
 'aland',
 'awater',
 'low_kg_consumed_Apples',
 'high_kg_consumed_Apples',
 'other_kg_consumed_Apples',
 'total_kg_consumed_Apples',
 'low_kg_consumed_Bananas',
 'high_kg_consumed_Bananas',
 'other_kg_consumed_Bananas',
 'total_kg_consumed_Bananas',
 'low_kg_consumed_Berries',
 'high_kg_consumed_Berries',
 'other_kg_consumed_Berries',
 'total_kg_consumed_Berries',
 'low_kg_consumed_Grapes

In [168]:
# set geometry of scaled_production to be the geometry column
scaled_production = scaled_production.set_geometry('geometry')
scaled_production.set_crs(epsg=4326, inplace=True)
scaled_production.head(10).explore()

In [169]:
scaled_production.to_file("./output-data/county-population-consumption-production-scaled-with-impacts.geojson", driver='GeoJSON')

In [170]:
!tippecanoe -zg -o ./output-data/county-population-consumption-production-scaled-with-impacts.mbtiles --detect-shared-borders --coalesce-densest-as-needed --extend-zooms-if-still-dropping ./output-data/county-population-consumption-production-scaled-with-impacts.geojson

For layer 0, using name "countypopulationconsumptionproductionscaledwithimpacts"
3108 features, 14113612 bytes of geometry, 5102955 bytes of string pool
Choosing a maxzoom of -z1 for features typically 141317 feet (43074 meters) apart, and at least 37841 feet (11534 meters) apart
Choosing a maxzoom of -z7 for resolution of about 2877 feet (877 meters) within features
tile 0/0/0 size is 5434110 with detail 12, >500000    
Going to try keeping the sparsest 8.28% of the features to make it fit
tile 0/0/0 size is 511258 with detail 12, >500000    
Going to try keeping the sparsest 7.29% of the features to make it fit
tile 1/0/0 size is 5480763 with detail 12, >500000    
Going to try keeping the sparsest 8.21% of the features to make it fit
tile 1/0/0 size is 518202 with detail 12, >500000    
Going to try keeping the sparsest 7.13% of the features to make it fit
tile 2/0/1 size is 3147170 with detail 12, >500000    
Going to try keeping the sparsest 14.30% of the features to make it fit
t